In [1]:
# %pip install --upgrade --quiet  langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pathlib
import textwrap
from langchain_google_genai import GoogleGenerativeAI

import google.generativeai as googleai
from dotenv import load_dotenv
from IPython.display import display
from IPython.display import Markdown
import os

_ = load_dotenv()

In [3]:
def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))


api_key = os.getenv("GOOGLE_API_KEY")
# googleai.configure(api_key=google_api)

In [4]:
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key)

In [5]:
response = llm.invoke(
    "What are some of the pros and cons of Python as a programming language?"
)

In [7]:
to_markdown(response)

> **Pros of Python**
> 
> * **Easy to learn:** Python is a very easy language to learn, even for beginners. It has a simple syntax and is very readable, making it easy to understand what the code is doing.
> * **Versatile:** Python is a very versatile language that can be used for a wide variety of tasks, including web development, data science, and machine learning.
> * **Open-source:** Python is an open-source language, which means that it is free to use and modify. This makes it a great option for developers who want to be able to control the code that they are using.
> * **Large community:** Python has a large and active community of developers who are constantly creating new libraries and tools. This makes it easy to find help and resources when you are learning or using Python.
> 
> **Cons of Python**
> 
> * **Slow:** Python is not as fast as some other programming languages, such as C++ or Java. This can be a problem for applications that require a lot of processing power.
> * **Not as performant:** Python is not as performant as some other programming languages, such as C++ or Java. This can be a problem for applications that require a lot of speed.
> * **Dynamic typing:** Python is a dynamically typed language, which means that the type of a variable is not checked until it is assigned a value. This can lead to errors if the type of a variable is not explicitly specified.
> 
> Overall, Python is a very good programming language for beginners and experienced developers alike. It is easy to learn, versatile, and has a large community of support. However, it is not as fast or performant as some other programming languages.

## Creating chain usign Gemini